# Setup 

## Import packages 

In [1]:
# General 
import os 
import numpy as np
import pandas as pd 
# For running bash scripts from inside python ... 
import subprocess
# For manipulating string objects 
import re
# for generating any necessary directories
import pathlib 
# For manipulating list objects 
import itertools 

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# For working with sequence objects 
from Bio.Seq import Seq

In [5]:
# For fetching sequences from Entrez 
from Bio import Entrez
from Bio import SeqIO

In [6]:
# For extracting features 
from Bio.SeqFeature import SeqFeature, FeatureLocation
# For creating SeqRecord objects 
from Bio.SeqRecord import SeqRecord

In [7]:
# For running R cells 
%load_ext rpy2.ipython

# Functions 

In [8]:
import sys
# Importing module of personal functions 
sys.path.append('/home/user1/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code')
from kCellReadR import * 

## Misc

In [101]:
speciesName = 'Mouse'
geneName = 'Fosb'

Entrez.email = "kehaliwoldemichael@gmail.com"  # Always tell NCBI who you are

# Sequence

## Loading sequences 

In [102]:
ensembl_transcriptIDs = return_ensemblTranscriptIDs(speciesName, geneName)
ensembl_transcriptIDs

['ENSMUST00000003640',
 'ENSMUST00000207334',
 'ENSMUST00000207716',
 'ENSMUST00000208230',
 'ENSMUST00000208326',
 'ENSMUST00000208446',
 'ENSMUST00000208505']

In [103]:
variantTable = table_transcriptsInfo(ensembl_transcriptIDs)
variantTable

,TranscriptNum,TranscriptID,TranscriptName,Assembly,Type,AA_Length,Is_Canonical
0,1,ENSMUST00000003640,Fosb-201,GRCm39,protein_coding,338,True
1,2,ENSMUST00000207334,Fosb-202,GRCm39,protein_coding,302,False
2,3,ENSMUST00000207716,Fosb-203,GRCm39,protein_coding,263,False
3,4,ENSMUST00000208230,Fosb-204,GRCm39,retained_intron,no protein,False
4,5,ENSMUST00000208326,Fosb-205,GRCm39,protein_coding,299,False
5,6,ENSMUST00000208446,Fosb-206,GRCm39,protein_coding,237,False
6,7,ENSMUST00000208505,Fosb-207,GRCm39,protein_coding,201,False


In [104]:
# Chooses canonical transcript
spliceVariant = int(variantTable[variantTable['Is_Canonical'] == 'True']['TranscriptNum'])
spliceVariant

1

In [105]:
geneName

'Fosb'

In [106]:
speciesName

'Mouse'

In [107]:
rC_exon_records, C_exon_records, CDS, cDNA, genomic = load_referenceSequences(speciesName, geneName, spliceVariant)

## Selecting sensor 

In [108]:
len(genomic[0])

7356

In [109]:
len(CDS[0])

1017

In [110]:
# Print lenth exons 
for exon in rC_exon_records:
    print(len(str(exon.seq)))

746
321
108
2607


In [111]:
len(rC_exon_records)

4

In [112]:
##### Initial if want to check variable length sesRNAs 
for i in range(192, 300):
    if(i%3 == 0):
        print(i)

192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297


In [113]:
# Longer the sesRNA ... the better it binds ... but the higher likelihood to introduce stop codons and ATGs 
# Want around 60 bp distance between multiple TAGs ... 
sesRNA_length = 192
parameters = parameters_sesRNA(speciesName, geneName,  spliceVariant, 'Reverse', sesRNA_length, 1, 1, 'None', 40, 70, 40, 10)

In [114]:
parameters.print_parameters()

[instance attributes]
species = Mouse
gene = Fosb
spliceVariant = 1
seqDirection = Reverse
length = 192
num_inF_TGG = 1
num_inF_Stop = 1
inF_ATG = None
minGC = 40
maxGC = 70
nearCenter = 40
fromStop = 10


In [115]:
rC_CDS = [CDS[0].reverse_complement()]
rC_genomic = [genomic[0].reverse_complement()]

In [116]:
sesRNA_length/2

96.0

In [117]:
all_sesRNAs, all_sequenceMetrics, all_sesRNA_objs = generate_all_sesRNAs(rC_exon_records, C_exon_records, CDS, parameters, variantTable['Type'])
all_sequenceMetrics

1
2
4


,sesNum,TypeSeq,SeqNum,ExonFrac,ExonProtFrac,CDSFrac,CDNAFrac,StartSeq,StopSeq,firstTGG,centralTGG,second_cTGG,numTGG,numTTGG,numTGGA,numTTGGA,numATG,numStop,gcCont
0,1,Reverse,2,7/7,6/6,6/6,7/7,2.0,194.0,24.0,69.0,60.0,6.0,0.0,0.0,0.0,0.0,1.0,62.0
1,2,Reverse,2,7/7,6/6,6/6,7/7,5.0,197.0,21.0,66.0,57.0,6.0,0.0,0.0,0.0,0.0,1.0,63.0
2,3,Reverse,2,7/7,6/6,6/6,7/7,8.0,200.0,18.0,63.0,132.0,6.0,0.0,0.0,0.0,0.0,1.0,63.5
3,4,Reverse,2,7/7,6/6,6/6,7/7,11.0,203.0,15.0,129.0,60.0,6.0,0.0,0.0,0.0,0.0,1.0,64.1
4,5,Reverse,2,5/7,4/6,6/6,7/7,14.0,206.0,12.0,126.0,57.0,6.0,0.0,0.0,0.0,0.0,1.0,63.0
5,6,Reverse,2,5/7,4/6,6/6,7/7,17.0,209.0,9.0,123.0,54.0,6.0,0.0,0.0,0.0,0.0,1.0,63.5
6,7,Reverse,2,5/7,4/6,4/6,5/7,20.0,212.0,6.0,120.0,51.0,6.0,0.0,0.0,0.0,0.0,1.0,64.1
7,8,Reverse,2,5/7,4/6,4/6,5/7,23.0,215.0,3.0,117.0,48.0,7.0,0.0,0.0,0.0,0.0,0.0,64.6
8,9,Reverse,2,5/7,4/6,4/6,5/7,26.0,218.0,0.0,114.0,45.0,7.0,0.0,1.0,0.0,0.0,0.0,63.5
9,10,Reverse,2,5/7,4/6,4/6,5/7,29.0,221.0,3.0,111.0,42.0,6.0,0.0,1.0,0.0,0.0,0.0,63.0


# Secondary structure 

## Working 

In [ ]:
generate_RNApred(all_sesRNAs, all_sequenceMetrics, geneName, 1)

In [ ]:
def generate_mfeProb(sequenceMetrics, species, spliceVariant): 
    rnaFold_prob = []

    spliceVariant = str(spliceVariant)
    save_speciesName = species.replace(" ", "_")
    gene_BasePath = ensembl_BasePath + '/' + save_speciesName + '/' + geneName 
    CDS_fileName = gene_BasePath + '_cds_' + spliceVariant + '_' + save_speciesName + '.fasta'
    
    pathTemp = '/home/user1/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/Output/BioPython/Temp'
    pathOutTempFold = pathTemp + '/temp.out'

    # sorting files in output of scandir 
    for entry in sorted(os.scandir(pathTemp), key=lambda e: e.name):
        # Defining command for RNAfold 
        commandFold = 'RNAfold -p -d2 --noLP < ' + entry.path + ' > ' + pathOutTempFold    
        # Generating RNAfold predictions 
        generateProb = subprocess.run(commandFold, shell=True, stdout=subprocess.PIPE)
        
        # Moving to Temp directory to work on fasta files 
        currentWD = os.getcwd()
        os.chdir('/home/user1/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/Output/BioPython/Temp')

        # Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
        readProb = subprocess.Popen("rnaFold_prob.sh", shell=True, stdout=subprocess.PIPE)
        returnedProb = readProb.stdout.read()
        # Waiting for last command to finish before storing value in temp.out file 
        readProb.wait()
        # Append frequences ... convert to percentage 
        rnaFold_prob.append(round(float(returnedProb)*100, 3))
        
        # Removing temp.out after finishing each run 
        os.system('rm -rf temp.out')
        os.system('rm -rf temp.csv')
        # Return to initial working directory 
        os.chdir(currentWD)

    # Removing files generated by RNAfold 
    os.system('rm -rf *ss.ps')
    os.system('rm -rf *dp.ps')
    
    # Adding RNA fold mfe ensemble frequency to sequenceMetrics 
    sequenceMetrics['mfeFreq'] = rnaFold_prob
    
    return sequenceMetrics  

In [ ]:
fold_sequenceMetrics = generate_mfeProb(all_sequenceMetrics, speciesName, spliceVariant)
fold_sequenceMetrics 

In [ ]:
def output_RIblast(sequenceMetrics, geneName, testSpecies, spliceVariant, targetName):
    spliceVariant = str(spliceVariant)
    save_test_speciesName = testSpecies.replace(" ", "_")
    gene_BasePath = ensembl_BasePath + '/' + save_test_speciesName + '/' + geneName 
    
    if targetName == 'CDS':
        target_fileName = gene_BasePath + '_cds_' + spliceVariant + '_' + save_test_speciesName + '.fasta'
    elif targetName == 'cDNA':
        target_fileName = gene_BasePath + '_cdna_' + spliceVariant + '_' + save_test_speciesName + '.fasta'
    elif targetName == 'genomic':
        target_fileName = gene_BasePath + '_genomic_' + spliceVariant + '_' + save_test_speciesName + '.fasta'
    print(target_fileName)
    
    # Path to tempRIblast folder 
    path_tempRIblast = '/home/user1/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/Output/RIblast/'
    query_Name = path_tempRIblast + geneName + '_db'
    
    # Generating query database 
    commandQuery = 'RIblast db -i ' + target_fileName + ' -o ' + query_Name
    os.system(commandQuery)

    # Path and file name for output CSV 
    outputName = path_tempRIblast + geneName + '.csv'
    # Path to directory sesRNA files 
    path_sesRNAs = '/home/user1/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/Output/BioPython/Temp'
    
    
    # Generating pd.DataFrame for storing calculated values 
    columns_RIblast = [' Accessibility Energy', ' Hybridization Energy', ' Interaction Energy', ' BasePair', 
                       ' Accessibility Energy', ' Hybridization Energy', ' Interaction Energy', ' BasePair']
    useful_RIblast =  pd.DataFrame(columns = columns_RIblast)
    
    # Iteratively generating calculations for sesRNA-target interaction 
    # Made sure to go through sesRNA files in order 
    for entry in sorted(os.scandir(path_sesRNAs), key=lambda e: e.name):
        # Running RIblast calculations 
        commandSearch = 'RIblast ris -i ' + entry.path + ' -o ' + outputName + ' -d ' + query_Name
        os.system(commandSearch)
        print(entry.path)
        
        # Remove first two lines from CVS to allow for parsing into pandas.Dataframe 
        commandRemoveLines = 'sed -i 1,2d ' + outputName 
        os.system(commandRemoveLines)

        outputCSV =  pd.read_csv(outputName, skiprows=[1])
        # Sorting by hybirzation energy ... have to have extra white space before column name 
        sorted_outputCSV = outputCSV.sort_values(' Hybridization Energy')

        topHybridizationE = sorted_outputCSV[[' Accessibility Energy', ' Hybridization Energy', ' Interaction Energy', ' BasePair']].iloc[0:1]
        secondHybridizationE = sorted_outputCSV[[' Accessibility Energy', ' Hybridization Energy', ' Interaction Energy', ' BasePair']].iloc[1:2]
        temp_RIblast_ouput = pd.concat([topHybridizationE.reset_index(drop=True), secondHybridizationE.reset_index(drop=True)], axis = 1)

        # Appending calculations for current sesRNA values 
        useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)
        
        # Clearing csv 
        command_clearCSV = 'rm -rf ' + outputName
        os.system(command_clearCSV)
        # Clear BioPython temp fasta file for sesRNA 
        os.system('rm -rf ' + entry.path)
    
    # Clear RIblast Temp directory 
    os.system("rm -rf " + path_tempRIblast + "*")
    
    return useful_RIblast

In [ ]:
#test_speciesName = 'Crab-eating macaque'
test_speciesName = speciesName

In [ ]:
# Downloading species name if test species is not same as initial species 
if test_speciesName != speciesName:
    test_ensembl_transcriptIDs = return_ensemblTranscriptIDs(test_speciesName, geneName)
    test_variantTable = table_transcriptsInfo(test_ensembl_transcriptIDs)
    # Chooses canonical transcript
    test_spliceVariant = int(test_variantTable[test_variantTable['Is_Canonical'] == 'True']['TranscriptNum'])
    test_rC_exon_records, test_C_exon_records, test_CDS, test_cDNA, test_genomic = load_referenceSequences(test_speciesName, geneName, test_spliceVariant)

In [ ]:
outputRIblast = output_RIblast(fold_sequenceMetrics, geneName, test_speciesName, spliceVariant, 'cDNA')
outputRIblast

In [ ]:
metricsTable_higherOrder = pd.concat([fold_sequenceMetrics.reset_index(drop=True), outputRIblast.reset_index(drop=True).iloc[:, 0:4]], axis = 1)

In [ ]:
sesRNA_length/2

In [ ]:
metricsTable_higherOrder

In [ ]:
sesChoice = 20
chosenSequence = all_sesRNAs[sesChoice - 1]

In [ ]:
str(chosenSequence)

In [ ]:
return_inFrame(chosenSequence, 'all')

In [ ]:
from Bio.Restriction import *

In [ ]:
len(EcoRI.search(chosenSequence)) == 0

In [ ]:
len(AscI.search(chosenSequence)) == 0

In [ ]:
len(HindIII.search(chosenSequence)) == 0

In [ ]:
testSeq = 'CCTCCTCGCTGCCCTCGGACTTGAGGATGTCCATCTGCAGCCCTTGCCGATGCTCCATGTCCAGGTCGTCGCAGTGGGCGAAGCCCACCGCCTCCTCGTCGGTGGCCGCCTGGAAGCCCATCCTGGCGAACATGCCGCTCATCTTCGCCTGGGATTTGTGCGACACCGAGGTGGCCACGTTGGAGAGCTTGCTGCGGAGGAGGG'

In [ ]:
testSeq = 'CCTCCTCGCTGCCCTCGGACTTGAGGATGTCCATCTGCAGCCCTTGCCGATGCTCCATGTCCAGGTCGTCGCAGTGGGCGAAGCCCACCGCCTCCTCGTCGGTGGCCGCCTGGAAGCCCATCCTGGCGAACATGCCGCTCATCTTCGCCTGGGATTTGTGCGACACCGAGGTGGCCACGTTGGAGAGCTTGCTGCGGAGGAGGG'

In [ ]:
return_inFrame(Seq(testSeq), 'all')

In [ ]:
save_all_sesRNAs_DNA(all_sesRNAs, speciesName, geneName)

In [ ]:
AscI.site

In [ ]:
testSeq = 'GGTAGCTCGACTGGCTTCTACCTTTCGGGTACCTTCACGGGCTTTTCTTTCGGGGCGACACGCTGACGTGGACGAAGCCCGACGGGGCCGCCATATAGTAGCGGTAGTACTCGCCTGACCCTAAGACGTAAAGGAAGCCGTAGGCTACATTGGACCCTCACCGGTAGCACCTATACCAGTTATTGTCGTGGT'

In [ ]:
testSeq[::-1]

In [ ]:
check_inSearchSeq(all_sesRNAs[0], CDS, 'Complement')

In [ ]:
def output_intaRNA(sequenceMetrics):
    # Call RNAfold on each sequence of output 
    top_intarnaE = []

    intarna_columns = ['E', 'start1', 'end1', 'start2', 'end2']
    all_sorted_intarna = []
    useful_intarna =  pd.DataFrame(columns = intarna_columns)

    martBase = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/biomaRt/'
    martBasePath = martBase + species
    # Loading sequences for gene CDS
    CDS_fileName = martBasePath + '/CDS_' + geneName + '.fasta'
    
    pathTemp = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp'
    pathOutTempIntaRNA = pathTemp + '/temp.csv'

    # sorting files in output of scandir 
    for entry in sorted(os.scandir(pathTemp), key=lambda e: e.name):
        # For checking which file currently working on (not in order for some reason) 
        print(entry.path)

        # Defining command for IntaRNA
        numThreads = 10
        commandInta = 'IntaRNA -t ' + CDS_fileName + ' -q ' + entry.path + ' --threads ' + str(numThreads) + ' --outMode=C > ' + pathOutTempIntaRNA
        # Generating IntaRNA predictions 
        os.system(commandInta)
        # Reading in csv file with information 
        intarnaOutput = pd.read_csv(pathOutTempIntaRNA, sep = ';')
        # Saving pd output as list of lists ... would be a pain to compute multiple times if desired 
        sorted_intarna = intarnaOutput.sort_values('E', ascending=True)
        # Concatinating select columuns of dataframe for inclusion into sequence metrics 
        all_sorted_intarna.append(sorted_intarna)
        print(sorted_intarna)
        # Getting first row ... with highest energy values ... and appending to DataFrame 
        useful_intarna = useful_intarna.append(sorted_intarna[intarna_columns].iloc[0:1])
        
        # Moving to Temp directory to work on fasta files 
        currentWD = os.getcwd()
        os.chdir('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp')

        # Removing temp.out after finishing each run 
        os.system('rm -rf temp.out')
        os.system('rm -rf temp.csv')
        # Return to initial working directory 
        os.chdir(currentWD)

    # Removing files generated by RNAfold 
    os.system('rm -rf *ss.ps')
    os.system('rm -rf *dp.ps')
    # Removing temp fasta files 
#     resetCommand = 'cd ' + pathTemp + ' && rm -rf *'
#     os.system(resetCommand)
    
    # Have to reset index since useful_intarna is slice of many pd.DataFrames 
    out_sequenceMetrics = pd.concat([sequenceMetrics.reset_index(), useful_intarna.reset_index()], axis = 1)
    
    return out_sequenceMetrics 

In [ ]:
intarna_sequenceMetrics = output_intaRNA(fold_sequenceMetrics)
intarna_sequenceMetrics

In [ ]:
def output_RNApred(sequenceMetrics):
    # Call RNAfold on each sequence of output 
    rnaFold_prob = []
    top_intarnaE = []

    intarna_columns = ['E', 'start1', 'end1', 'start2', 'end2']
    all_sorted_intarna = []
    useful_intarna =  pd.DataFrame(columns = intarna_columns)

    martBase = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/biomaRt/'
    martBasePath = martBase + species
    # Loading sequences for gene CDS
    CDS_fileName = martBasePath + '/CDS_' + geneName + '.fasta'
    
    pathTemp = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp'
    pathOutTempFold = pathTemp + '/temp.out'
    pathOutTempIntaRNA = pathTemp + '/temp.csv'

    # sorting files in output of scandir 
    for entry in sorted(os.scandir(pathTemp), key=lambda e: e.name):
        # For checking which file currently working on (not in order for some reason) 
        print(entry.path)

        # Defining command for RNAfold 
        commandFold = 'RNAfold -p -d2 --noLP < ' + entry.path + ' > ' + pathOutTempFold    
        # Generating RNAfold predictions 
        generateProb = subprocess.run(commandFold, shell=True, stdout=subprocess.PIPE)

        # Defining command for IntaRNA
        numThreads = 10
        commandInta = 'IntaRNA -t ' + CDS_fileName + ' -q ' + entry.path + ' --threads ' + str(numThreads) + ' --outMode=C > ' + pathOutTempIntaRNA
        # Generating IntaRNA predictions 
        os.system(commandInta)
        # Reading in csv file with information 
        intarnaOutput = pd.read_csv(pathOutTempIntaRNA, sep = ';')
        # Saving pd output as list of lists ... would be a pain to compute multiple times if desired 
        sorted_intarna = intarnaOutput.sort_values('E', ascending=True)
        # Concatinating select columuns of dataframe for inclusion into sequence metrics 
        all_sorted_intarna.append(sorted_intarna)
        # Getting first row ... with highest energy values ... and appending to DataFrame 
        useful_intarna = useful_intarna.append(sorted_intarna[intarna_columns].iloc[0:1])
        
        # Moving to Temp directory to work on fasta files 
        currentWD = os.getcwd()
        os.chdir('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp')

        # Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
        readProb = subprocess.Popen("rnaFold_prob.sh", shell=True, stdout=subprocess.PIPE)
        returnedProb = readProb.stdout.read()
        # Waiting for last command to finish before storing value in temp.out file 
        readProb.wait()
        # Append frequences ... convert to percentage 
        rnaFold_prob.append(float(returnedProb)*100)

        
        # Removing temp.out after finishing each run 
        os.system('rm -rf temp.out')
        os.system('rm -rf temp.csv')
        # Return to initial working directory 
        os.chdir(currentWD)

    # Removing files generated by RNAfold 
    os.system('rm -rf *ss.ps')
    os.system('rm -rf *dp.ps')
    # Removing temp fasta files 
    resetCommand = 'cd ' + pathTemp + ' && rm -rf *'
    os.system(resetCommand)
    
    # Adding RNA fold mfe ensemble frequency to sequenceMetrics 
    sequenceMetrics['mfeFreq'] = rnaFold_prob
    # Have to reset index since useful_intarna is slice of many pd.DataFrames 
    out_sequenceMetrics = pd.concat([sequenceMetrics.reset_index(), useful_intarna.reset_index()], axis = 1)
    
    return out_sequenceMetrics 

In [ ]:
higherSequenceMetrics = output_RNApred(all_sequenceMetrics)
higherSequenceMetrics

# Outputing sesRNA

In [ ]:
testSeq = Seq('TGGGAGTAGTGGTGGTAATGA')
testStr = str(testSeq)

In [ ]:
testStr

In [ ]:
convert_DNA(testSeq, 1)

In [ ]:
# Test that all and numbering is working 
convert_DNA(testSeq, 3) == convert_DNA(testSeq, 'All')

In [ ]:
convert_DNA(rC_multiExon_sesRNAs[0], 'All')

In [ ]:
# Testing that continious reading frame being produced if number set to 0 
len(convert_DNA(rC_multiExon_sesRNAs[0], 0).translate(to_stop = True)) == len(rC_multiExon_sesRNAs[0])/3

In [ ]:
# Checking that right number of TGG being converted 
return_inFrame(convert_DNA(rC_multiExon_sesRNAs[0], 'All').back_transcribe(), 'numTGG')

In [ ]:
convert_DNA(testSeq, 1) - (len(testSeq)/2)

In [ ]:
abs(convert_DNA(testSeq, 1) - (len(testSeq)/2))

In [ ]:
np.sort(abs(convert_DNA(testSeq, 1) - (len(testSeq)/2))) + (len(testSeq)/2)

In [ ]:
np.array(sorted(convert_DNA(testSeq, 1) - (len(testSeq)/2), key = abs)) + (len(testSeq)/2)

In [ ]:
# Function for saving both original template DNA sequence and the converted RNA given list of sesRNAs Seq objects
# Could change later to make for separate output files ... have not decided yet whether to just put them together into one labeled fasta file 
def save_sesRNAs(sequences_sesRNAs, geneName, version, numConvertTGG):
    # Generating BioPython directory if does not exist 
    pathlib.Path('Output/BioPython').mkdir(parents=True, exist_ok=True)
    
    # Defining save name and description for outputs (original DNA and converted RNA)
    DNA_outputID = geneName + '_sesRNA'
    DNA_outputDescription = "sesRNA DNA original for " + geneName
    convertedRNA_outputID = geneName + '_sesRNA'
    convertedRNA_outputDescription = "sesRNA converted RNA for " + geneName
    
    # Creating empty list for storing sequences 
    outputSeqMulti_DNA = []
    outputSeqMulti_convertedRNA= []
    
    # Generating SeqRecord objects in preparation for writing FASTA files ... edited description to include number of sequence
    i = 1
    for sequence in sequences_sesRNAs:
        outputSeqMulti_DNA.append(SeqRecord(sequence, id = DNA_outputID+str(i), description = DNA_outputDescription))
        outputSeqMulti_convertedRNA.append(SeqRecord(convert_DNA(sequence, numberConvertTGG), id = convertedRNA_outputID+str(i), description = convertedRNA_outputDescription))
        i += 1
        
    # Write output fasta files 
    DNA_outputName = "Output/BioPython/" + geneName + "_sesRNA_" + version + ".fasta" 
    convertedRNA_outputName = "Output/BioPython/" + geneName + "_sesRNA_convertedRNA_" + version + ".fasta" 
    
    with open(outputName, "w") as output_handle:
        # Writing original template DNA files 
        SeqIO.write(outputSeqMulti_DNA, output_handle, "fasta")
        # Writing orginal template RNA files 
        SeqIO.write(outputSeqMulti_convertedRNA, output_handle, "fasta")

# Not working or not being used 

In [ ]:
seqTdT = 'ttacttgtacagctcgtccatgccgtacaggaacaggtggtggcggccctcggagcgctcgtactgttccacgatggtgtagtcctcgttgtgggaggtgatgtccagcttggtgtccacgtagtagtagccgggcagttgcacgggcttcttggccatgtagatggtcttgaactccaccaggtagtggccgccgtccttcagcttcagggcctggtggatctcgcccttcagcacgccgtcgcgggggtacaggcgctcggtggaggcctcccagcccatggtcttcttctgcattacggggccgtcgggggggaagttggtgccgcgcatcttcaccttgtagatcagcgtgccgtcctgcagggaggagtcctgggtcacggtcaccagaccgccgtcctcgaagttcatcacgcgctcccacttgaagccctcggggaaggacagcttcttgtaatcggggatgtcggcggggtgcttcacgtacgccttggagccgtacatgaactggggggacaggatgtcccaggcgaagggcagggggccgcccttggtcaccttcagcttggcggtctgggtgccctcgtaggggcggccctcgccctcgccctcgatctcgaactcgtggccgttcatggagccctccatgcgcaccttgaagcgcatgaactctttgatgacggccatgttgttgtcctcggaggaggcggtgccggagctgccgctgccggtgctgccggtgccatgccccaggaacaggtggtggcggccctcggagcgctcgtactgttccacgatggtgtagtcctcgttgtgggaggtgatgtccagcttggtgtccacgtagtagtagccgggcagttgcacgggcttcttggccatgtagatggtcttgaactccaccaggtagtggccgccgtccttcagcttcagggcctggtggatctcgcccttcagcacgccgtcgcgggggtacaggcgctcggtggaggcctcccagcccatggtcttcttctgcattacggggccgtcgggggggaagttggtgccgcgcatcttcaccttgtagatcagcgtgccgtcctgcagggaggagtcctgggtcacggtcaccagaccgccgtcctcgaagttcatcacgcgctcccacttgaagccctcggggaaggacagcttcttgtaatcggggatgtcggcggggtgcttcacgtacgccttggagccgtacatgaactggggggacaggatgtcccaggcgaagggcagggggccgcccttggtcaccttcagcttggcggtctgggtgccctcgtaggggcggccctcgccctcgccctcgatctcgaactcgtggccgttcatggagccctccatgcgcaccttgaagcgcatgaactctttgatgacctcctcgcccttgctcaccat'
seqTdT = seqTdT.upper()
seqTdT

In [ ]:
len(Seq(seqTdT).translate()) == len(seqTdT)/3

In [ ]:
num_inF_TGG, num_inF_TTGG, num_inF_TGGA, num_inF_TTGGA, num_inF_ATG, num_inF_Stop, indices_inF_TGG, \
    indices_inF_ATG, indices_inF_Stop = \
    return_inFrame(Seq(seqTdT).reverse_complement(), 'all')

In [ ]:
num_inF_TGG

In [ ]:
num_inF_TTGG

In [ ]:
num_inF_TGGA

In [ ]:
num_inF_TTGGA

In [ ]:
# Generating pd.Dataframe
df = pd.DataFrame(all_sequenceMetrics)
# Converting DataFrame to json and dumping it to std.out
df_json = df.reset_index().to_json(orient="values")


In [ ]:
df_json

In [ ]:
# Initial if want to check variable length sesRNAs 
for i in range(200, 300):
    if(i%3 == 0):
        print(i)

In [ ]:
additional_sesRNA = 'gagaggaagggcagaggcaccaacccgggcggaggaggaggcgcggcggcggcggcggctcagaccccctccccggcccgcatctgtgcagctttccgggcgatgccagaatagatgccggggcaatgtcccgccgcaaacagggcaacccgcagcacttgtcccagagggaactcatcacgcgtaagtgtctgccgcacgcgcgaagggccggcggctggggctccgggcgcccggggctgg'

In [ ]:
len(additional_sesRNA)

In [ ]:
all_sequenceMetrics = all_sequenceMetrics.append(all_sequenceMetrics.iloc[11, :])

In [ ]:
all_sequenceMetrics

In [ ]:
len(all_sesRNAs)

In [ ]:
all_sesRNAs.append(Seq(additional_sesRNA.upper()))

In [ ]:
len(all_sesRNAs)

In [ ]:
all_sesRNAs

In [ ]:
sys.path.append("/usr/share/ViennaRNA")

In [ ]:
# Leading RNAfold as RNA 
import sys
sys.path.append("/usr/lib/python3.9/site-packages/RNA")
import _RNA as RNA

In [ ]:
import pyseqlib

In [ ]:
from pyseqlib import pyRNAfold

In [ ]:
pyRNAfold i

In [ ]:
pyseqlib.fold_compound(sequence)

In [ ]:
RNA.fold_compound(sequence)

In [ ]:
sys.path.append("/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Packages/RNA")
import _RNA as RNA

In [ ]:
sys.path.append("/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions")
import RNA_Fold

In [ ]:
md = RNA.md()

In [ ]:
import example

In [ ]:
sequence = "CGCAGGGAUACCCGCG"
 
# create new fold_compound object
fc = RNA.fold_compound(sequence)
 
# compute minimum free energy (mfe) and corresponding structure
(ss, mfe) = fc.mfe()
 

In [ ]:
# The RNA sequence
seq = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA"
 
# compute minimum free energy (MFE) and corresponding structure
(ss, mfe) = RNA.fold(seq)

In [ ]:
import pickle 

In [ ]:
mfe

In [ ]:
pathOuput = "/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/seqObject.p"
pickle.dump(testSeq, open(pathOuput, 'wb'))

In [ ]:
# Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
pathFuncPython = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/'
# command = 'python ' + '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/RNA_Fold.py'
# command = 'cd ' + pathFuncPython + ' | ./RNA_Fold.py'
# command = 'cd ' + pathFuncPython + ' | ls -a'

command = 'ls -a'
readProb = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
returnedProb = readProb.stdout.read()

In [ ]:
returnedProb

In [ ]:
command 

In [ ]:
str(rC_multiExon_sesRNAs[0])

In [ ]:
testSeq = str(rC_multiExon_sesRNAs[0])
testSeq

In [ ]:
RNA.fold(testSeq)

In [ ]:
def compute_mfeFreq(sequence):
    # create a fold_compound object for the current sequence
    fc = RNA.fold_compound(sequence)

    # compute the MFE and corresponding structure
    (mfe_struct, mfe) = fc.mfe()


    # compute partition function
    (bp_propensity, dG) = fc.pf()

    # compute frequency of MFE structure (the 'hard' way)
    kT = RNA.exp_param().kT / 1000.

    prob_mfe = math.exp((dG - mfe) / kT)
    
    return prob_mfe

In [ ]:
rC_multi_startSeq

In [ ]:
rC_multiExon_sesRNAs

In [ ]:
str(rC_multiExon_sesRNAs[8])

In [ ]:
str(rC_multiExon_sesRNAs[18])

In [ ]:
from seqfold import dg, dg_cache, fold

In [ ]:
# just returns minimum free energy
dg("GGGAGGTCGTTACATCTGGGTAACACCGGTACTGATCCGGTGACCTCCC", temp = 37.0)  # -12.94

# Test Intarna (test strength of binding, off-target, ...)  

In [ ]:
import intarnapvalue

In [ ]:
testSeq = str(rC_multiExon_sesRNAs[0])
testSeq

In [ ]:
target = str(CDS[0].seq)
target 

In [ ]:
from intarnapvalue.intarna_pvalue import IntaRNApvalue

In [ ]:
?IntaRNApvalue

In [ ]:
?intarna_pvalue

In [ ]:
IntaRNApvalue(['--query', testSeq, '--target', target])

In [ ]:
IntaRNApvalue['-q', 'AGGAUG', '-t', 'UUUAUCGUU', '-s', '10', '-m', 'b', '-d', 'gauss', '--threads', '3']

In [ ]:
command = 'python -m intarnapvalue --query GCUGAAAAACAUAACCCAUAAAAUGCUAGCUGUACCAGGAACCA --target GGUUUCUUCGCCUCUGCGUUCACCAAAGUGUUCACCC -s 10 --shuffle-mode b --threads 0' 

In [ ]:
readProb = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
readProb.wait()
returnedProb = readProb.stdout.read()
returnedProb

In [ ]:
# Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
readProb = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
returnedProb = readProb.stdout.read()
# Waiting for last command to finish before storing value in temp.out file 
readProb.wait()

In [ ]:
pd.read_csv('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/output.csv', sep = ';')

In [ ]:
pd.read_csv('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/output.csv', sep = ';')

In [ ]:
pd.read_csv('/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Functions/output.csv', sep = ';')

In [ ]:
# Trying to load entire transcripome ... obviously failed ... 
allRat_fileName = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Data/Sequences/Reference/Ensembl/All/Rattus_norvegicus.Rnor_6.0.cdna.all.fa'
allRat = list(SeqIO.parse(allRat_fileName, "fasta"))

In [ ]:
python -m intarnapvalue --query GCUGAAAAACAUAACCCAUAAAAUGCUAGCUGUACCAGGAACCA --target GGUUUCUUCGCCUCUGCGUUCACCAAAGUGUUCACCC --scores 10000 --shuffle-mode b --threads 0

In [ ]:
intarnapvalue --query GCUGAAAAACAUAACCCAUAAAAUGCUAGCUGUACCAGGAACCA --target GGUUUCUUCGCCUCUGCGUUCACCAAAGUGUUCACCC --scores 10000 --shuffle-mode b --threads 0

# Misc

In [ ]:
# For returning index of findings 
searchCodon = 'TAG'
[m.start() for m in re.finditer(searchCodon, str(seq))]

In [ ]:
testSeq = sesRNAs[0]
testSeq

In [ ]:
lastATG = [m.start() for m in re.finditer('ATG', str(testSeq))][-1]
lastTGG = [m.start() for m in re.finditer('TGG', str(testSeq))][-1]

In [ ]:
lastTGG

In [ ]:
lastATG

In [ ]:
lastATG < lastTGG

In [ ]:
searchCodon = 'ATG'
[m.start() for m in re.finditer(searchCodon, str(testSeq))][-1]

In [ ]:
seq.count('TAG') < 4

In [ ]:
seq[0:100].count('TAG')

In [ ]:
stopCodons = ['TAG', 'TAA', 'TGA']
stopCodons 

In [ ]:
indiciesTGG

In [ ]:
indiciesStop

In [ ]:
length = 200 
center = length/2

In [ ]:
arrayStop = np.array(indiciesStop)
arrayIndicies = np.array(indiciesTGG) 
centralTGGs = arrayIndicies[abs(arrayIndicies - center) < 10]

In [ ]:
centralTGGs

In [ ]:
np.in1d(centralTGGs,arrayStop)

In [ ]:
# Check if array contains values that are within range of values in another array 
any((min(abs(arrayStop - i)) > 10) for i in centralTGGs)

In [ ]:
centralTGGs

In [ ]:
indiciesStop

In [ ]:
testStop = [90, 16, 174]

In [ ]:
(min(abs(arrayStop - centralTGGs[0])) > 10)

In [ ]:
min(abs(arrayStop - centralTGGs[0])) > 10

In [ ]:
centeralTGGs = offset.min()
centeralTGGs

In [ ]:
centralTGGs = np.all(offset == offset.min())
centralTGGs

In [ ]:
centeralTGGs = np.where(offset == offset.min())
centeralTGG

In [ ]:
offset = abs(arrayIndicies - center) 
centerTGG = indiciesTGG[np.argmin(offset)]

In [ ]:
any(abs(x - centerTGG) < 10 for x in indiciesStop)

In [ ]:
indiciesStop = []
for codons in stopCodons:
    indiciesStop.extend([m.start() for m in re.finditer(codons, str(testSeq))])

In [ ]:
len(indiciesStop)

In [ ]:
[m.start() for m in re.finditer('TGA', str(testSeq))]

In [ ]:
[m.start() for m in re.finditer('TAA', str(testSeq))]

In [ ]:
[m.start() for m in re.finditer('TAG', str(testSeq))]

In [ ]:
testSeq.count(stopCodons)

In [ ]:
indiciesTGG

In [ ]:
len(indiciesTGG)

In [ ]:
testSeq = sesRNAs[0]

In [ ]:
lastTGG = [m.start() for m in re.finditer('TGG', str(testSeq))][-1]

In [ ]:
testSeq

In [ ]:
indiciesTGG = [m.start() for m in re.finditer('TGG', str(testSeq))]

In [ ]:
start = 0 
stop = 200

In [ ]:
middle = (start + stop) / 2

In [ ]:
abs(middle - indiciesTGG[0])

In [ ]:
type(indiciesTGG)

In [ ]:
indiciesTGG

In [ ]:
any(indiciesTGG) > 2

In [ ]:
length = 200 

In [ ]:
any(abs(x - (length/2)) < 20 for x in indiciesTGG)

In [ ]:
testList = [50, 60, 170, 200]

In [ ]:
any(abs(x - (length/2)) < 10 for x in testList)

In [ ]:
testSeq

In [ ]:
os.path.isdir('Output/BioPython')

In [ ]:
outputFileName = os.getcwd() + ''

In [ ]:
testSeq